In [13]:
import pdb
import argparse
import numpy as np
import numpy.random as npr
import time
import os
import sys
import pickle
import ssl 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.backends.cudnn as cudnn
from torch.optim.lr_scheduler import MultiStepLR

from torchvision.utils import make_grid
from torchvision import datasets, transforms

from Cutout.util.misc import CSVLogger
from Cutout.util.cutout import Cutout

from Cutout.model.resnet import ResNet18
from Cutout.model.wide_resnet import WideResNet

In [ ]:
def train(args, model, device, trainset, model_optimizer, epoch,
          example_stats):
    train_loss = 0.
    correct = 0.
    total = 0.

    model.train()

    # Get permutation to shuffle trainset
    trainset_permutation_inds = npr.permutation(
        np.arange(len(trainset.targets)))

    print('\n=> Training Epoch #%d' % (epoch))

    batch_size = args['batch_size']
    for batch_idx, batch_start_ind in enumerate(
            range(0, len(trainset.targets), batch_size)):

        # Get trainset indices for batch
        batch_inds = trainset_permutation_inds[batch_start_ind:
                                               batch_start_ind + batch_size]

        # Get batch inputs and targets, transform them appropriately
        transformed_trainset = []
        for ind in batch_inds:
            transformed_trainset.append(trainset.__getitem__(ind)[0])
        inputs = torch.stack(transformed_trainset)
        targets = torch.LongTensor(
            np.array(trainset.targets)[batch_inds].tolist())

        # Map to available device
        inputs, targets = inputs.to(device), targets.to(device)

        # Forward propagation, compute loss, get predictions
        model_optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        _, predicted = torch.max(outputs.data, 1)

        # Update statistics and loss
        acc = predicted == targets
        for j, index in enumerate(batch_inds):

            # Get index in original dataset (not sorted by forgetting)
            index_in_original_dataset = train_indx[index]

            # Compute missclassification margin
            output_correct_class = outputs.data[j, targets[j].item()]
            sorted_output, _ = torch.sort(outputs.data[j, :])
            if acc[j]:
                # Example classified correctly, highest incorrect class is 2nd largest output
                output_highest_incorrect_class = sorted_output[-2]
            else:
                # Example misclassified, highest incorrect class is max output
                output_highest_incorrect_class = sorted_output[-1]
            margin = output_correct_class.item(
            ) - output_highest_incorrect_class.item()

            # Add the statistics of the current training example to dictionary
            index_stats = example_stats.get(index_in_original_dataset,
                                            [[], [], []])
            index_stats[0].append(loss[j].item())
            index_stats[1].append(acc[j].sum().item())
            index_stats[2].append(margin)
            example_stats[index_in_original_dataset] = index_stats

        # Update loss, backward propagate, update optimizer
        loss = loss.mean()
        train_loss += loss.item()
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        loss.backward()
        model_optimizer.step()

        sys.stdout.write('\r')
        sys.stdout.write(
            '| Epoch [%3d/%3d] Iter[%3d/%3d]\t\tLoss: %.4f Acc@1: %.3f%%' %
            (epoch, args['epochs'], batch_idx + 1,
             (len(trainset) // batch_size) + 1, loss.item(),
             100. * correct.item() / total))
        sys.stdout.flush()

        # Add training accuracy to dict
        index_stats = example_stats.get('train', [[], []])
        index_stats[1].append(100. * correct.item() / float(total))
        example_stats['train'] = index_stats

In [ ]:
#define args 
args = {
    'dataset': 'cifar10', 
    'model': 'resnet18', 
    'batch_size': 128, 
    'epochs': 200, 
    'learning_rate': 0.1, 
    'data_augmentation': True, 
    'cutout': False, 
    'cuda':False,
    'n_holes': 1, 
    'length': 16, 
    'no_cuda': False, 
    'seed': 1, 
    'sorting_file': 'none', 
    'remove_n': 0, 
    'keep_lowest_n': 0, 
    'remove_subsample': 0, 
    'noise_percent_labels': 0, 
    'noise_percent_pixels': 0, 
    'noise_std_pixels': 0, 
    'optimizer': 'sgd', 
    'input_dir': 'cifar10_results/', 
    'output_dir': 'cifar10_results',
    'data_augmentation': False
 }

Dataset Processing

In [29]:
#prevents ssl certificate errors 
ssl._create_default_https_context = ssl._create_unverified_context

# Image Preprocessing
normalize = transforms.Normalize(
    mean=[x / 255.0 for x in [125.3, 123.0, 113.9]],
    std=[x / 255.0 for x in [63.0, 62.1, 66.7]])

# define train transform 
train_transform = transforms.Compose([])
if args['data_augmentation']:
    train_transform.transforms.append(transforms.RandomCrop(32, padding=4))
    train_transform.transforms.append(transforms.RandomHorizontalFlip())
train_transform.transforms.append(transforms.ToTensor())
train_transform.transforms.append(normalize)
if args['cutout']:
    train_transform.transforms.append(
        Cutout(n_holes=args.n_holes, length=args.length))

test_transform = transforms.Compose([transforms.ToTensor(), normalize])

# Check for output dir 
os.makedirs(args['output_dir'], exist_ok=True)

# Load the appropriate train and test datasets
if args['dataset'] == 'cifar10':
    num_classes = 10
    train_dataset = datasets.CIFAR10(
        root='/tmp/data/',
        train=True,
        transform=train_transform,
        download=True)

    test_dataset = datasets.CIFAR10(
        root='/tmp/data/',
        train=False,
        transform=test_transform,
        download=True)
elif args['dataset'] == 'cifar100':
    num_classes = 100
    train_dataset = datasets.CIFAR100(
        root='/tmp/data/',
        train=True,
        transform=train_transform,
        download=True)

    test_dataset = datasets.CIFAR100(
        root='/tmp/data/',
        train=False,
        transform=test_transform,
        download=True)

train_indx = npr.permutation(np.arange(len(
        train_dataset.targets)))[:len(train_dataset.targets) -
                                      0]

train_dataset.data = train_dataset.data[train_indx, :, :, :]
train_dataset.targets = np.array(
    train_dataset.targets)[train_indx].tolist()

print('Trainset of ' + str(len(train_dataset.targets)) + ' examples')


Files already downloaded and verified
Files already downloaded and verified
Trainset of 50000 examples


Model Setup

In [38]:
#device settings
args['cuda'] = args['cuda'] and torch.cuda.is_available()
use_cuda = args['cuda']
device = torch.device("cuda" if use_cuda else "cpu")
#cudnn.benchmark = True  # Should make training go faster for large models

# Setup model
if args['model'] == 'resnet18':
    model = ResNet18(num_classes=num_classes)
elif args['model'] == 'wideresnet':
        model = WideResNet(
            depth=28, num_classes=num_classes, widen_factor=10, dropRate=0.3)
else:
    print(
        'Specified model not recognized. Options are: resnet18 and wideresnet')
    
# Setup loss
#model = model.cuda()
criterion = nn.CrossEntropyLoss()#.cuda()
criterion.__init__(reduce=False)

# Setup optimizer
if args['optimizer'] == 'adam':
    model_optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
elif args['optimizer'] == 'sgd':
    model_optimizer = torch.optim.SGD(
        model.parameters(),
        lr=args['learning_rate'],
        momentum=0.9,
        nesterov=True,
        weight_decay=5e-4)
    scheduler = MultiStepLR(
        model_optimizer, milestones=[60, 120, 160], gamma=0.2)
else:
    print('Specified optimizer not recognized. Options are: adam and sgd')

KeyError: 'cuda'

Train

In [ ]:
example_stats = {}

best_acc = 0
elapsed_time = 0
for epoch in range(args['epochs']):
    start_time = time.time()

    train(args, model, device, train_dataset, model_optimizer, epoch,
          example_stats)
    test(epoch, model, device, example_stats)

    epoch_time = time.time() - start_time
    elapsed_time += epoch_time
    print('| Elapsed time : %d:%02d:%02d' % (get_hms(elapsed_time)))

    # Update optimizer step
    if args.optimizer == 'sgd':
        scheduler.step(epoch)

    # Save the stats dictionary
    fname = os.path.join(args.output_dir, save_fname)
    with open(fname + "__stats_dict.pkl", "wb") as f:
        pickle.dump(example_stats, f)

    # Log the best train and test accuracy so far
    with open(fname + "__best_acc.txt", "w") as f:
        f.write('train test \n')
        f.write(str(max(example_stats['train'][1])))
        f.write(' ')
        f.write(str(max(example_stats['test'][1])))


In [ ]:
train_dataset.targets

[6,
 9,
 9,
 4,
 1,
 1,
 2,
 7,
 8,
 3,
 4,
 7,
 7,
 2,
 9,
 9,
 9,
 3,
 2,
 6,
 4,
 3,
 6,
 6,
 2,
 6,
 3,
 5,
 4,
 0,
 0,
 9,
 1,
 3,
 4,
 0,
 3,
 7,
 3,
 3,
 5,
 2,
 2,
 7,
 1,
 1,
 1,
 2,
 2,
 0,
 9,
 5,
 7,
 9,
 2,
 2,
 5,
 2,
 4,
 3,
 1,
 1,
 8,
 2,
 1,
 1,
 4,
 9,
 7,
 8,
 5,
 9,
 6,
 7,
 3,
 1,
 9,
 0,
 3,
 1,
 3,
 5,
 4,
 5,
 7,
 7,
 4,
 7,
 9,
 4,
 2,
 3,
 8,
 0,
 1,
 6,
 1,
 1,
 4,
 1,
 8,
 3,
 9,
 6,
 6,
 1,
 8,
 5,
 2,
 9,
 9,
 8,
 1,
 7,
 7,
 0,
 0,
 6,
 9,
 1,
 2,
 2,
 9,
 2,
 6,
 6,
 1,
 9,
 5,
 0,
 4,
 7,
 6,
 7,
 1,
 8,
 1,
 1,
 2,
 8,
 1,
 3,
 3,
 6,
 2,
 4,
 9,
 9,
 5,
 4,
 3,
 6,
 7,
 4,
 6,
 8,
 5,
 5,
 4,
 3,
 1,
 8,
 4,
 7,
 6,
 0,
 9,
 5,
 1,
 3,
 8,
 2,
 7,
 5,
 3,
 4,
 1,
 5,
 7,
 0,
 4,
 7,
 5,
 5,
 1,
 0,
 9,
 6,
 9,
 0,
 8,
 7,
 8,
 8,
 2,
 5,
 2,
 3,
 5,
 0,
 6,
 1,
 9,
 3,
 6,
 9,
 1,
 3,
 9,
 6,
 6,
 7,
 1,
 0,
 9,
 5,
 8,
 5,
 2,
 9,
 0,
 8,
 8,
 0,
 6,
 9,
 1,
 1,
 6,
 3,
 7,
 6,
 6,
 0,
 6,
 6,
 1,
 7,
 1,
 5,
 8,
 3,
 6,
 6,
 8,
 6,
 8,
 4,
 6,
 6,
